In [44]:
import pandas as pd
from myUtils import pickle_load, pickle_dump
from yf_utils import split_train_val_test, random_slices, lookback_slices
from yf_utils import top_set_sym_freq_cnt, get_grp_top_syms_n_freq
from yf_utils import eval_grp_top_syms_n_freq


pd.set_option("display.max_rows", 10)
pd.set_option("display.max_columns", 12)
pd.set_option("display.max_colwidth", 12)
pd.set_option("display.width", 160)  # code-runner format

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

fp_df_close_clean = "df_close_clean"

In [45]:
#######################################################################
## SELECT RUN PARAMETERS.async Parameters can also be passed using papermill by running yf_7_freq_cnt_pm_.ipynb
verbose = True  # True prints more output
# verbose = False  # True prints more output

# write run results to df_eval_results
# store_results = False
store_results = True

# select run type
run_type = "train"
# run_type = 'validate'
# run_type = "test"

# number of max lookback tuples to create for iloc iloc_start_train:iloc_end_train:iloc_end_eval
# i.e. number of grp_top_set_syms_n_freq and grp_top_set_syms
# n_samples = 400
n_samples = 2

# for training, the number of days to lookback from iloc max-lookback iloc_end_train
# days_lookbacks = [15, 30, 60]
# days_lookbacks = [30, 60, 120]
days_lookbacks = [15, 30, 60, 120]
days_lookbacks.sort()

# number of days in dataframe to evaluate effectiveness of the training, days_eval = len(df_eval)
#  days_eval = 4 means buy at close on 1st day after the signal, hold for 2nd and 3rd day, sell at close on 4th day
days_eval = 4
# days_eval = 5

# number of the most-common symbols from days_lookbacks' performance rankings to keep
n_top_syms = 20
# n_top_syms = 5

# slice starts and ends for selecting the best performing symbols
syms_start = 0
syms_end = 10


##########################################################
# fp_df_eval_results = f"df_eval_results_{run_type}"
fp_df_eval_results = "test_df_eval"
##########################################################
#######################################################################

In [46]:

print(f"verbose : {verbose }")
print(f"store_results: {store_results}")
print(f"run_type: {run_type}")
print(f"n_samples: {n_samples}")
print(f"days_lookbacks: {days_lookbacks}")
print(f"days_eval: {days_eval}")
print(f"n_top_syms: {n_top_syms}")
print(f"syms_start: {syms_start}")
print(f"syms_end: {syms_end}")
print(f"fp_df_eval_results: {fp_df_eval_results}")

df_close_clean = pickle_load(path_data_dump, fp_df_close_clean)

# Split df_close_clean into training (df_train), validation (df_val) and test (df_test) set.
# The default split is 0.7, 0.2, 0.1 respectively.
df_train, df_val, df_test = split_train_val_test(df_close_clean)

max_days_lookbacks = max(days_lookbacks)
print(f"max_days_lookbacks: {max_days_lookbacks}")

# Load df according to run_type
if run_type == "train":
    df = df_train.copy()
elif run_type == "validate":
    df = df_val.copy()
elif run_type == "test":
    df = df_test.copy()
else:
    msg_stop = f"ERROR run_type must be 'train', 'validate', or 'test', run_type is: {run_type}"
    raise SystemExit(msg_stop)

# Print dataframe for the run, and lengths of other dataframes
print(f"run_type: {run_type}, df.tail(3):\n{df.tail(3)}\n")
len_df = len(df)
len_df_train = len(df_train)
len_df_val = len(df_val)
len_df_test = len(df_test)
print(f"run_type: {run_type}, len(df): {len(df)}")
print(
    f"len_df_train: {len_df_train}, len_df_val: {len_df_val}, len_df_test: {len_df_test}"
)

# return n_samples slices
max_lookback_slices = random_slices(
    len_df,
    n_samples=n_samples,
    days_lookback=max(days_lookbacks),
    days_eval=days_eval,
    verbose=False,
)
# return n_samples * len(days_lookbacks) slices
sets_lookback_slices = lookback_slices(
    max_slices=max_lookback_slices, days_lookbacks=days_lookbacks, verbose=False
)

if verbose:
    print(f"number of max_lookback_slices is equal to n_samples = {n_samples}")
    print(f"max_lookback_slices:\n{max_lookback_slices}\n")
    print(f"number of sets in sets_lookback_slices is equal to n_samples = {n_samples}")
    print(f"sets_lookback_slices:\n{sets_lookback_slices}\n")
    print(f"days_lookbacks: {days_lookbacks}")
    print(
        f'number of tuples in each "set of lookback slices" is equal to len(days_lookbacks): {len(days_lookbacks)}'
    )

# #### Generate grp_top_set_syms_n_freq. It is a list of sub-lists, e.g.:
#  - [[('AGY', 7), ('PCG', 7), ('KDN', 6), ..., ('CYT', 3)], ..., [('FCN', 9), ('HIG', 9), ('SJR', 8), ..., ('BFH', 2)]]
# #### grp_top_set_syms_n_freq has n_samples sub-lists. Each sub-list corresponds to a tuple in the max_lookback_slices. Each sub-list has n_top_syms tuples of (symbol, frequency) pairs, and is sorted in descending order of frequency. The frequency is the number of times the symbol appears in the top n_top_syms performance rankings of CAGR/UI, CAGR/retnStd and retnStd/UI.
# #### Therefore, symbols in the sub-list are the best performing symbols for the periods in days_lookbacks. Each sub-list corresponds to a tuple in max_lookback_slices. There are as many sub-lists as there are tuples in max_lookback_slices.
grp_top_set_syms_n_freq, grp_top_set_syms, dates_end_df_train = get_grp_top_syms_n_freq(
    df, sets_lookback_slices, days_lookbacks, n_top_syms, syms_start, syms_end, verbose
)

# #### print the best performing symbols for each set in sets_lookback_slices
for i, top_set_syms_n_freq in enumerate(grp_top_set_syms_n_freq):
    l_sym_freq_cnt = top_set_sym_freq_cnt(top_set_syms_n_freq)
    if verbose:
        print(f"max_lookback_slices:             {max_lookback_slices}")
        # print(f'set_lookback_slices: {sets_lookback_slices[i]}\n')
        print(
            f"set_lookback_slices {i + 1} of {len(sets_lookback_slices):>3}:    {sets_lookback_slices[i]}\n"
        )
        print(f"max_days_lookbacks:              {max_days_lookbacks}")
        print(f"df end date for days_lookbacks:  {dates_end_df_train[i]}")
        print(f"days_lookbacks:                  {days_lookbacks}")
        print(f"sym_freq_15:                     {l_sym_freq_cnt[0]}")
        print(f"sym_freq_14:                     {l_sym_freq_cnt[1]}")
        print(f"sym_freq_13:                     {l_sym_freq_cnt[2]}")
        print(f"sym_freq_12:                     {l_sym_freq_cnt[3]}")
        print(f"sym_freq_11:                     {l_sym_freq_cnt[4]}")
        print(f"sym_freq_10:                     {l_sym_freq_cnt[5]}")
        print(f"sym_freq_9:                      {l_sym_freq_cnt[6]}")
        print(f"sym_freq_8:                      {l_sym_freq_cnt[7]}")
        print(f"sym_freq_7:                      {l_sym_freq_cnt[8]}")
        print(f"sym_freq_6:                      {l_sym_freq_cnt[9]}")
        print(f"sym_freq_5:                      {l_sym_freq_cnt[10]}")
        print(f"sym_freq_4:                      {l_sym_freq_cnt[11]}")
        print(f"sym_freq_3:                      {l_sym_freq_cnt[12]}")
        print(f"sym_freq_2:                      {l_sym_freq_cnt[13]}\n")


# #### Evaluate performance of symbols in set_lookback_slices versus SPY
l_row_add_total = eval_grp_top_syms_n_freq(
    df, max_lookback_slices, sets_lookback_slices, grp_top_set_syms_n_freq, days_lookbacks, days_eval, n_samples, n_top_syms, syms_start, syms_end, verbose
)

for row_add_total in l_row_add_total:
    print(f"row_add_total: {row_add_total}")

if store_results:  # record results to df
    df_eval_results = pickle_load(path_data_dump, fp_df_eval_results)
    print(f"df_eval_results BEFORW store results:\n{df_eval_results}\n")

    for row_add_total in l_row_add_total:
        # print(f'row_add_total: {row_add_total}')
        df_eval_results.loc[len(df_eval_results)] = row_add_total
        print(f"appended row_add_total to df_eval_results:\n{row_add_total}\n")

    pickle_dump(df_eval_results, path_data_dump, fp_df_eval_results)
    print(f"Save results to: {fp_df_eval_results}")
    df_eval_results = pickle_load(path_data_dump, fp_df_eval_results)
    print(f"df_eval_results AFTER store results:\n{df_eval_results}\n")


verbose : True
store_results: True
run_type: train
n_samples: 2
days_lookbacks: [15, 30, 60, 120]
days_eval: 4
n_top_syms: 20
syms_start: 0
syms_end: 10
fp_df_eval_results: test_df_eval
max_days_lookbacks: 120
run_type: train, df.tail(3):
                     A         AA        AAL         AAP        AAPL         AB  ...          ZG       ZION        ZTO         ZTS       ZUMZ        ZWS
Date                                                                             ...                                                                    
2021-06-15  143.119141  35.880054  22.790001  190.966675  128.315628  39.616650  ...  111.209999  51.980713  30.783392  183.434158  45.570000  50.638294
2021-06-16  142.099823  34.960556  22.830000  187.407333  128.820419  39.607914  ...  108.839996  52.562309  30.199211  181.892944  45.959999  51.023483
2021-06-17  145.395294  33.200668  22.230000  185.943817  130.443665  39.005150  ...  113.660004  49.215782  29.733847  184.194885  44.130001  48.998

In [60]:
_keys = ['df_eval_results', 'df', 'max_lookback_slices']
_vals = [df_eval_results, df, max_lookback_slices]


In [62]:
_d = dict(zip(_keys, _vals))

In [64]:
_l_keys = list(_d.keys())
_l_vals = list(_d.values())
print(_l_keys)
print(_l_vals)

['df_eval_results', 'df', 'max_lookback_slices']
[   n_samples     days_lookbacks  days_eval  n_top_syms  syms_start  syms_end  sym_freq_cnt  grp(CAGR)_mean  grp(CAGR)_std  grp(CAGR)_mean/std  grp(CAGR/UI)_mean  grp(CAGR/UI)_std  grp(CAGR/UI)_mean/std  grp(CAGR/retnStd)_mean  grp(CAGR/retnStd)_std  grp(CAGR/retnStd)_mean/std  grp(retnStd/UI)_mean  grp(retnStd/UI)_std  grp(retnStd/UI)_mean/std  SPY_CAGR   SPY_CAGR/UI  SPY_CAGR/retnStd  SPY_retnStd/UI
0          2  [15, 30, 60, 120]          4          20           0        10             7    1.518831e+03   2.606535e+03            0.582701       5.203492e+05      8.960229e+05             0.580732           4.765758e+04            8.154898e+04               0.584404                    3.987393              4.378174             0.910743       0.112420  8.372475e+01         54.015054        1.550026
1          2  [15, 30, 60, 120]          4          20           0        10             6   -4.571495e-01   5.374084e-01           -0.850656 

In [106]:
yf_test_values = [
  df_eval_results,
  df,
  max_lookback_slices,
  sets_lookback_slices,
  grp_top_set_syms_n_freq,
  days_lookbacks,
  days_eval,
  n_samples,
  n_top_syms,
  syms_start,
  syms_end,
  verbose]

yf_test_keys = [
  'df_eval_results',
  'df',
  'max_lookback_slices',
  'sets_lookback_slices',
  'grp_top_set_syms_n_freq,
  'days_lookbacks',
  'days_eval',
  'n_samples',
  'n_top_syms',
  'syms_start',
  'syms_end',
  'verbose']



In [110]:
for val in eval(yf_test_var_values):
  print(eval(val))

TypeError: eval() arg 1 must be a string, bytes or code object

In [69]:
yf_test_keys = [str(val) for val in yf_test_vals]
print(type(yf_test_keys))
# yf_test_keys[2]

'[(572, 692, 696), (59, 179, 183)]'

In [56]:
yf_test[1]

,A,AA,AAL,AAP,AAPL,AB,ABB,ABBV,ABC,ABM,ABR,ABT,ACGL,ACHC,ACIW,...,XRAY,XRX,XYL,YELP,YUMC,YY,ZBH,ZBRA,ZD,ZG,ZION,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-04-18,50.734451,31.401224,42.429222,132.239700,33.128162,13.696466,18.135284,49.198269,74.836243,38.130112,4.969970,39.409283,31.576668,42.779999,21.160000,...,60.059582,21.645435,45.710377,32.860001,31.161926,39.467556,110.811676,89.889999,69.184227,35.330002,34.190353,11.961975,51.636120,16.500000,21.610952
2017-04-19,50.772724,30.946419,43.000633,133.656754,33.006157,13.666758,18.200665,49.043682,74.192696,38.130112,4.952813,39.554462,31.716667,42.730000,21.330000,...,60.272251,21.398764,45.896454,32.830002,31.399723,40.414566,111.451164,90.410004,69.773438,35.799999,34.319107,12.065994,51.925400,16.850000,21.659666
2017-04-20,50.830112,31.678062,43.794777,137.542023,33.419090,13.696466,18.740068,49.298748,74.664040,38.469845,4.935656,39.908367,31.709999,42.840000,21.490000,...,60.591255,21.491262,46.724518,33.430000,31.466312,41.181183,112.924690,90.820000,70.802452,35.950001,35.263355,12.122731,52.349674,17.799999,22.088379
2017-04-21,50.964058,31.341906,43.726997,137.804779,33.379200,13.607337,19.058800,49.329662,73.657906,38.505604,4.947093,39.500031,31.663334,43.080002,21.500000,...,60.446259,21.337093,47.124584,33.490002,31.152411,41.379608,112.313019,90.970001,70.271339,36.160000,35.065918,12.302398,52.253254,18.200001,22.068893
2017-04-24,52.083416,32.933723,44.976330,138.048828,33.700642,13.518205,19.557339,49.978947,73.367851,38.925785,4.952813,40.044483,31.873333,43.560001,21.469999,...,61.219604,21.768772,47.887501,33.889999,31.694609,41.451756,114.240677,93.019997,71.084595,36.480000,36.018761,12.368589,53.024654,18.400000,22.702221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-11,142.040451,36.957741,23.530001,189.805603,126.049034,40.097115,32.719875,107.528770,117.128136,46.971684,16.241077,106.788467,39.290001,67.800003,39.340000,...,65.457428,22.567350,115.789093,40.720001,68.215134,66.735954,151.358368,508.480011,112.243477,113.470001,52.695789,31.288363,180.005920,46.939999,50.638294
2021-06-14,142.891541,36.532593,22.990000,188.996658,129.147064,39.756416,32.795635,107.510139,116.043274,46.283630,16.521683,107.342270,39.139999,66.750000,39.020000,...,64.558968,21.799328,114.916939,40.570000,67.959358,67.871117,150.660294,509.920013,112.191307,116.599998,51.504002,31.219053,182.357269,46.009998,50.440754
2021-06-15,143.119141,35.880054,22.790001,190.966675,128.315628,39.616650,32.975571,107.910744,116.336479,46.516220,15.713878,107.274261,39.529999,65.709999,39.029999,...,65.776878,21.924353,115.720505,39.900002,66.946060,65.857132,151.415741,506.100006,113.278259,111.209999,51.980713,30.783392,183.434158,45.570000,50.638294


In [47]:
df

,A,AA,AAL,AAP,AAPL,AB,...,ZG,ZION,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,
2017-04-18,50.734451,31.401224,42.429222,132.239700,33.128162,13.696466,...,35.330002,34.190353,11.961975,51.636120,16.500000,21.610952
2017-04-19,50.772724,30.946419,43.000633,133.656754,33.006157,13.666758,...,35.799999,34.319107,12.065994,51.925400,16.850000,21.659666
2017-04-20,50.830112,31.678062,43.794777,137.542023,33.419090,13.696466,...,35.950001,35.263355,12.122731,52.349674,17.799999,22.088379
2017-04-21,50.964058,31.341906,43.726997,137.804779,33.379200,13.607337,...,36.160000,35.065918,12.302398,52.253254,18.200001,22.068893
2017-04-24,52.083416,32.933723,44.976330,138.048828,33.700642,13.518205,...,36.480000,36.018761,12.368589,53.024654,18.400000,22.702221
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-11,142.040451,36.957741,23.530001,189.805603,126.049034,40.097115,...,113.470001,52.695789,31.288363,180.005920,46.939999,50.638294
2021-06-14,142.891541,36.532593,22.990000,188.996658,129.147064,39.756416,...,116.599998,51.504002,31.219053,182.357269,46.009998,50.440754
2021-06-15,143.119141,35.880054,22.790001,190.966675,128.315628,39.616650,...,111.209999,51.980713,30.783392,183.434158,45.570000,50.638294


In [48]:
max_lookback_slices

[(572, 692, 696), (59, 179, 183)]

In [49]:
sets_lookback_slices

[[(677, 692, 696), (662, 692, 696), (632, 692, 696), (572, 692, 696)],
 [(164, 179, 183), (149, 179, 183), (119, 179, 183), (59, 179, 183)]]

In [51]:
pd.set_option("display.max_rows", 10)
pd.set_option("display.max_columns", 30)
pd.set_option("display.max_colwidth", 20)
pd.set_option("display.width", 500)  # code-runner format
print(f"df_eval_results AFTER store results:\n{df_eval_results}\n")

df_eval_results AFTER store results:
   n_samples     days_lookbacks  days_eval  n_top_syms  syms_start  syms_end  sym_freq_cnt  grp(CAGR)_mean  grp(CAGR)_std  grp(CAGR)_mean/std  grp(CAGR/UI)_mean  grp(CAGR/UI)_std  grp(CAGR/UI)_mean/std  grp(CAGR/retnStd)_mean  grp(CAGR/retnStd)_std  grp(CAGR/retnStd)_mean/std  grp(retnStd/UI)_mean  grp(retnStd/UI)_std  grp(retnStd/UI)_mean/std  SPY_CAGR   SPY_CAGR/UI  SPY_CAGR/retnStd  SPY_retnStd/UI
0          2  [15, 30, 60, 120]          4          20           0        10             7    1.518831e+03   2.606535e+03            0.582701       5.203492e+05      8.960229e+05             0.580732           4.765758e+04            8.154898e+04               0.584404                    3.987393              4.378174             0.910743       0.112420  8.372475e+01         54.015054        1.550026
1          2  [15, 30, 60, 120]          4          20           0        10             6   -4.571495e-01   5.374084e-01           -0.850656      -8.26413

In [52]:
_df = df_eval_results.copy()
_df = _df[0:0]
_df

,n_samples,days_lookbacks,days_eval,n_top_syms,syms_start,syms_end,sym_freq_cnt,grp(CAGR)_mean,grp(CAGR)_std,grp(CAGR)_mean/std,grp(CAGR/UI)_mean,grp(CAGR/UI)_std,grp(CAGR/UI)_mean/std,grp(CAGR/retnStd)_mean,grp(CAGR/retnStd)_std,grp(CAGR/retnStd)_mean/std,grp(retnStd/UI)_mean,grp(retnStd/UI)_std,grp(retnStd/UI)_mean/std,SPY_CAGR,SPY_CAGR/UI,SPY_CAGR/retnStd,SPY_retnStd/UI


In [53]:
pickle_dump(_df, path_data_dump, fp_df_eval_results)
my_df = pickle_load(path_data_dump, fp_df_eval_results)
my_df

,n_samples,days_lookbacks,days_eval,n_top_syms,syms_start,syms_end,sym_freq_cnt,grp(CAGR)_mean,grp(CAGR)_std,grp(CAGR)_mean/std,grp(CAGR/UI)_mean,grp(CAGR/UI)_std,grp(CAGR/UI)_mean/std,grp(CAGR/retnStd)_mean,grp(CAGR/retnStd)_std,grp(CAGR/retnStd)_mean/std,grp(retnStd/UI)_mean,grp(retnStd/UI)_std,grp(retnStd/UI)_mean/std,SPY_CAGR,SPY_CAGR/UI,SPY_CAGR/retnStd,SPY_retnStd/UI
